In [14]:
import pandas as pd
import re


In [15]:
df=pd.read_excel('base semana 3.xlsx')
df

,How old are you?,What industry do you work in?,Job title,"If your job title needs additional context, please clarify here:","What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)","How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",Please indicate the currency,"If ""Other,"" please indicate the currency here:","If your income needs additional context, please provide it here:",What country do you work in?,"If you're in the U.S., what state do you work in?",What city do you work in?,How many years of professional work experience do you have overall?,How many years of professional work experience do you have in your field?,What is your highest level of education completed?,What is your gender?,What is your race? (Choose all that apply.)
0,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,55,0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,54.6,4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,34,NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,25-34,Nonprofits,Program Manager,NaN,62,3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,60,7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28185,18-24,study,no job,tking a course,0,0.0,Other,inr,no incime,india,NaN,kannur,1 year or less,1 year or less,High School,Man,Asian or Asian American
28186,18-24,Transport or Logistics,Apprentie Data et IA,appliquée au domaine du transport,19000,0.0,EUR,NaN,NaN,France,NaN,Paris,1 year or less,1 year or less,Master's degree,Woman,Black or African American
28187,35-44,Engineering or Manufacturing,Principal Engineer,NaN,151000,0.0,USD,NaN,NaN,United States,Iowa,Cedar Rapids,11 - 20 years,11 - 20 years,Master's degree,Woman,White
28188,55-64,Computing or Tech,Senior Technical Writer,NaN,125000,5000.0,CAD,NaN,NaN,Canada,NaN,Toronto,31 - 40 years,21 - 30 years,College degree,Woman,White


In [16]:
unique_values = df['What country do you work in?'].unique()


In [39]:
import re
import pandas as pd

# ==========================
# CONFIG
# ==========================

file_path = "base semana 3.xlsx"
output_path = "base_semana_3_cleaned_v3.xlsx"

column_name = "What country do you work in?"

import re
import pandas as pd

# ==========================
# HELPERS
# ==========================
def norm(s: str) -> str:
    s = str(s).strip().lower()
    s = re.sub(r"\s+", " ", s)
    # quitar algunos signos que estorban
    s = re.sub(r"[<>]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Frases / tokens que NO son países (se van a Other)
JUNK_RE = re.compile(
    r"(we don't get raises|bonus based|i earn commission|policy|currently finance|remote|global|international|europe|usd|\$|/year|salary|dbfemf|ibdia|loutreland|^ss$|^ff$|^1$|^y$|^na$|^n/a$)",
    re.IGNORECASE
)

# ==========================
# CORE: PATTERNS (orden importa)
# ==========================
PATTERNS = [
    # --- US / UK (los tuyos) ---
    ("United Kingdom", re.compile(
        r"(^|[^a-z])(u\s*\.?\s*k\.?|uk)([^a-z]|$)|united\s*kingdom|great\s*britain|britain|england|scotland|wales|northern\s*ireland|unites\s*kingdom|united\s*kindom|englang",
        re.IGNORECASE
    )),
    ("United States", re.compile(
        r"(^|[^a-z])(u\s*\.?\s*s\s*\.?\s*a?\.?|us|usa|usab|usaa|usat|uss|i\.s\.|u\.a\.|isa)([^a-z]|$)"
        r"|united\s*.*\s*sta|uni.{0,8}sta|united\s*states|america|🇺🇸|united\s*sates|unted\s*states|untied\s*states|united\s*sttes|united\s*sttes",
        re.IGNORECASE
    )),

    # --- Canada (fuzzy “c*nad*”) ---
    ("Canada", re.compile(r"(^|[^a-z])c\s*a\s*n\s*a\s*d\s*a\w*([^a-z]|$)", re.IGNORECASE)),

    # --- Netherlands ---
    ("Netherlands", re.compile(r"(the\s+netherlands|netherlands|nederland|(^|[^a-z])nl([^a-z]|$))", re.IGNORECASE)),

    # --- Australia ---
    ("Australia", re.compile(r"(australia|australi|australian)", re.IGNORECASE)),

    # --- Spain ---
    ("Spain", re.compile(r"(spain|españa|catalonia)", re.IGNORECASE)),  # si NO quieres catalonia -> bórralo

    # --- Finland / France / Germany / Ireland / India / Denmark ---
    ("Finland", re.compile(r"finland", re.IGNORECASE)),
    ("France", re.compile(r"france", re.IGNORECASE)),
    ("Germany", re.compile(r"germany|deutschland", re.IGNORECASE)),
    ("Ireland", re.compile(r"ireland", re.IGNORECASE)),
    ("India", re.compile(r"india", re.IGNORECASE)),
    ("Denmark", re.compile(r"denmark|danmark", re.IGNORECASE)),

    # --- Switzerland / Luxembourg / Belgium / Sweden / Norway ---
    ("Switzerland", re.compile(r"switzerland", re.IGNORECASE)),
    ("Luxembourg", re.compile(r"luxembourg|luxemburg", re.IGNORECASE)),
    ("Belgium", re.compile(r"belgium", re.IGNORECASE)),
    ("Sweden", re.compile(r"sweden", re.IGNORECASE)),
    ("Norway", re.compile(r"norway", re.IGNORECASE)),

    # --- Hong Kong (typos) ---
    ("Hong Kong", re.compile(r"(hong\s*kong|hong\s*konh|hong\s*kongkong)", re.IGNORECASE)),

    # --- Kuwait ---
    ("Kuwait", re.compile(r"kuwait", re.IGNORECASE)),

    # --- Sri Lanka ---
    ("Sri Lanka", re.compile(r"sri\s*lanka", re.IGNORECASE)),

    # --- Greece / Austria / Brazil ---
    ("Greece", re.compile(r"greece", re.IGNORECASE)),
    ("Austria", re.compile(r"austria", re.IGNORECASE)),
    ("Brazil", re.compile(r"brazil|brasil", re.IGNORECASE)),

    # --- Colombia ---
    ("Colombia", re.compile(r"colombia", re.IGNORECASE)),

    # --- New Zealand / NZ / Aotearoa ---
    ("New Zealand", re.compile(r"(new\s*zealand|(^|[^a-z])nz([^a-z]|$)|aotearoa)", re.IGNORECASE)),

    # --- Trinidad and Tobago / Cayman Islands / Bermuda / Bahamas / Isle of Man / Jersey ---
    ("Trinidad and Tobago", re.compile(r"trinidad\s*and\s*tobago", re.IGNORECASE)),
    ("Cayman Islands", re.compile(r"cayman\s*islands", re.IGNORECASE)),
    ("Bermuda", re.compile(r"bermuda", re.IGNORECASE)),
    ("Bahamas", re.compile(r"the\s*bahamas|bahamas", re.IGNORECASE)),
    ("Isle of Man", re.compile(r"isle\s*of\s*man", re.IGNORECASE)),
    ("Jersey", re.compile(r"jersey|channel\s*islands", re.IGNORECASE)),

    # --- Czech Republic / Czechia ---
    ("Czech Republic", re.compile(r"(czech\s*republic|czechia|česká\s*republika)", re.IGNORECASE)),

    # --- Latvia / Romania / Serbia / Philippines / Russia / Poland / Turkey / Slovakia / Portugal ---
    ("Latvia", re.compile(r"latvia", re.IGNORECASE)),
    ("Romania", re.compile(r"romania", re.IGNORECASE)),
    ("Serbia", re.compile(r"serbia", re.IGNORECASE)),
    ("Philippines", re.compile(r"philippines", re.IGNORECASE)),
    ("Russia", re.compile(r"russia", re.IGNORECASE)),
    ("Poland", re.compile(r"poland", re.IGNORECASE)),
    ("Turkey", re.compile(r"turkey", re.IGNORECASE)),
    ("Slovakia", re.compile(r"slovakia", re.IGNORECASE)),
    ("Portugal", re.compile(r"portugal", re.IGNORECASE)),

    # --- Puerto Rico ---
    ("Puerto Rico", re.compile(r"puerto\s*rico", re.IGNORECASE)),

    # --- Rwanda / Bangladesh / UAE ---
    ("Rwanda", re.compile(r"rwanda", re.IGNORECASE)),
    ("Bangladesh", re.compile(r"bangladesh", re.IGNORECASE)),
    ("United Arab Emirates", re.compile(r"(united\s*arab\s*emirates|(^|[^a-z])uae([^a-z]|$)|(^|[^a-z])ua([^a-z]|$))", re.IGNORECASE)),

    # --- China / Taiwan / Cambodia / Vietnam / Singapore / South Korea / Thailand / Indonesia / Malaysia ---
    ("China", re.compile(r"(china|mainland\s*china)", re.IGNORECASE)),
    ("Taiwan", re.compile(r"taiwan", re.IGNORECASE)),
    ("Cambodia", re.compile(r"cambodia", re.IGNORECASE)),
    ("Vietnam", re.compile(r"vietnam", re.IGNORECASE)),
    ("Singapore", re.compile(r"singapore", re.IGNORECASE)),
    ("South Korea", re.compile(r"south\s*korea", re.IGNORECASE)),
    ("Thailand", re.compile(r"thailand", re.IGNORECASE)),
    ("Indonesia", re.compile(r"indonesia", re.IGNORECASE)),
    ("Malaysia", re.compile(r"malaysia", re.IGNORECASE)),

    # --- Mexico / Panamá ---
    ("Mexico", re.compile(r"(mexico|méxico)", re.IGNORECASE)),
    ("Panama", re.compile(r"(panama|panamá)", re.IGNORECASE)),

    # --- Africa countries + others you listed ---
    ("South Africa", re.compile(r"south\s*africa", re.IGNORECASE)),
    ("Nigeria", re.compile(r"nigeria", re.IGNORECASE)),
    ("Congo", re.compile(r"congo", re.IGNORECASE)),
    ("Uganda", re.compile(r"uganda", re.IGNORECASE)),
    ("Morocco", re.compile(r"morocco", re.IGNORECASE)),
    ("Ecuador", re.compile(r"ecuador", re.IGNORECASE)),
    ("Zimbabwe", re.compile(r"zimbabwe", re.IGNORECASE)),
    ("Ghana", re.compile(r"ghana", re.IGNORECASE)),
    ("Eritrea", re.compile(r"eritrea", re.IGNORECASE)),
    ("Somalia", re.compile(r"somalia", re.IGNORECASE)),
    ("Sierra Leone", re.compile(r"sierra\s*leone", re.IGNORECASE)),
    ("Zambia", re.compile(r"zambia", re.IGNORECASE)),
    ("Liberia", re.compile(r"liberia", re.IGNORECASE)),

    # --- Europe misc ---
    ("Hungary", re.compile(r"hungary", re.IGNORECASE)),
    ("Italy", re.compile(r"italy|italia", re.IGNORECASE)),
    ("Slovenia", re.compile(r"slovenia", re.IGNORECASE)),
    ("Croatia", re.compile(r"croatia", re.IGNORECASE)),
    ("Ukraine", re.compile(r"ukraine", re.IGNORECASE)),
    ("Bulgaria", re.compile(r"bulgaria", re.IGNORECASE)),
    ("Estonia", re.compile(r"estonia", re.IGNORECASE)),
    ("Lithuania", re.compile(r"lithuania", re.IGNORECASE)),
    ("Cyprus", re.compile(r"cyprus", re.IGNORECASE)),
    ("Liechtenstein", re.compile(r"liechtenstein", re.IGNORECASE)),
    ("Bosnia and Herzegovina", re.compile(r"bosnia\s*and\s*herzegovina", re.IGNORECASE)),

    # --- Middle East / Asia misc ---
    ("Israel", re.compile(r"israel", re.IGNORECASE)),
    ("Afghanistan", re.compile(r"afghanistan", re.IGNORECASE)),
    ("Pakistan", re.compile(r"pakistan", re.IGNORECASE)),
    ("Jordan", re.compile(r"jordan", re.IGNORECASE)),
    ("Saudi Arabia", re.compile(r"saudi\s*arabia", re.IGNORECASE)),
    ("Qatar", re.compile(r"qatar", re.IGNORECASE)),
    ("Egypt", re.compile(r"egypt", re.IGNORECASE)),
    ("Myanmar", re.compile(r"(myanmar|burma)", re.IGNORECASE)),
    ("Tanzania", re.compile(r"tanzania", re.IGNORECASE)),

    # --- Americas misc ---
    ("Chile", re.compile(r"chile", re.IGNORECASE)),
    ("Uruguay", re.compile(r"uruguay", re.IGNORECASE)),
    ("Costa Rica", re.compile(r"costa\s*rica", re.IGNORECASE)),
    ("Jamaica", re.compile(r"jamaica", re.IGNORECASE)),

    # --- Japan / Argentina (como pediste) ---
    ("Japan", re.compile(r"japan", re.IGNORECASE)),
    ("Argentina", re.compile(r"argentina", re.IGNORECASE)),
]

# Manual override(s)
OVERRIDES = {
    "contracts": "United States",
    # agrega aquí los que quieras a mano
}

def clean_country(value):
    if pd.isna(value):
        return "Other"

    t = norm(value)

    # override manual
    if t in OVERRIDES:
        return OVERRIDES[t]

    # basura obvia -> Other
    if JUNK_RE.search(t):
        return "Other"

    for name, rx in PATTERNS:
        if rx.search(t):
            return name

    return "Other"



# ==========================
# RUN PIPELINE
# ==========================

df = pd.read_excel(file_path)

df["country_clean"] = df[column_name].apply(clean_country)

df.to_excel(output_path, index=False)

print("File saved successfully:")
print(output_path)


# ==========================
# QUICK VALIDATION
# ==========================

print("\nSample validation:")
print(df[[column_name, "country_clean"]].head(30))


File saved successfully:
base_semana_3_cleaned_v3.xlsx

Sample validation:
   What country do you work in?   country_clean
0                 United States   United States
1                United Kingdom  United Kingdom
2                            US   United States
3                           USA   United States
4                            US   United States
5                           USA   United States
6                           USA   United States
7                 United States   United States
8                            US   United States
9                 United States   United States
10                United States   United States
11                          USA   United States
12                United States   United States
13                United States   United States
14                       Canada          Canada
15               United Kingdom  United Kingdom
16                          USA   United States
17                          usa   United States
18           